In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285397 sha256=ba4ca6bd900efe4248081517b2adfe814194117b6e9a4b812559b42781ee37c8
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import when
import pyspark.sql.functions as sf
import os

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/log_content/'

In [ ]:
def etl_1_day(path ,file_name):
    #print('---------------------')
    #print('Read data from Kafka')
    #print('---------------------')
    df = spark.read.json(path+file_name)
    #print('---------------------')
    #print('Showing top 5')
    #print('---------------------')
    #df.show()
    #print('---------------------')
    #print('Showing data structure')
    #print('---------------------')
    #df.printSchema()
    #print('---------------------')
    #print('Showing data value')
    #print('---------------------')
    df = df.select('_source.*')
    # df.show()
    # print('---------------------')
    # print('ETL Start')
    # print('---------------------')
    df = df.withColumn("Type",
           when((col("AppName") == 'CHANNEL') | (col("AppName") =='DSHD')| (col("AppName") =='KPLUS')| (col("AppName") =='KPlus'), "Truyền Hình")
        .when((col("AppName") == 'VOD') | (col("AppName") =='FIMS_RES')| (col("AppName") =='BHD_RES')|
              (col("AppName") =='VOD_RES')| (col("AppName") =='FIMS')| (col("AppName") =='BHD')| (col("AppName") =='DANET'), "Phim Truyện")
        .when((col("AppName") == 'RELAX'), "Giải Trí")
        .when((col("AppName") == 'CHILD'), "Thiếu Nhi")
        .when((col("AppName") == 'SPORT'), "Thể Thao")
        .otherwise("Error"))
    # print('---------------------')
    # print('Showing new data structure')
    # print('---------------------')
    df = df.select('Contract','Type','TotalDuration')
    df = df.filter(df.Type != 'Error')
    # df.printSchema()
    # df.select('Contract','Type','TotalDuration').show(5)
    # df = df.groupBy('Contract','Type').sum()
    # df = df.withColumnRenamed('sum(TotalDuration)','TotalDuration')
    # df = df.cache()
    # print('---------------------')
    # print('Pivoting Data')
    # print('---------------------')
    # result = df.groupBy("Contract").pivot("Type").sum("TotalDuration")
    # print('---------------------')
    # print('Showing result')
    # print('---------------------')
    # result.show(5)
    print('Finished Processing {}'.format(file_name))
    return df

In [ ]:
list_file = os.listdir(path)
list_file.sort()
file_name = list_file[0]
result1 = etl_1_day(path ,file_name)
for i in list_file[1:]:
    file_name2 = i
    result2 = etl_1_day(path ,file_name2)
    result1 = result1.union(result2)
result1 = result1.groupby('Contract','Type').sum()
result1 = result1.withColumnRenamed('sum(TotalDuration)','TotalDuration')
final = result1.groupBy("Contract").pivot("Type").sum("TotalDuration")

Finished Processing 20220401.json
Finished Processing 20220402.json
Finished Processing 20220403.json
Finished Processing 20220404.json
Finished Processing 20220405.json
Finished Processing 20220406.json
Finished Processing 20220407.json
Finished Processing 20220408.json
Finished Processing 20220409.json
Finished Processing 20220410.json
Finished Processing 20220411.json
Finished Processing 20220412.json
Finished Processing 20220413.json
Finished Processing 20220414.json
Finished Processing 20220415.json


In [ ]:
#final.show(10)

+---------+--------+-----------+---------+--------+-----------+
| Contract|Giải Trí|Phim Truyện|Thiếu Nhi|Thể Thao|Truyền Hình|
+---------+--------+-----------+---------+--------+-----------+
|HTFD11598|    null|      14367|     null|    null|      16104|
|HPFD48556|      69|       null|     null|    null|     595153|
|NBFD10014|    null|       null|     null|    null|     704933|
|HNH619088|    null|      77275|     3657|    null|     546772|
|HNH036174|    null|      37842|     null|    null|     172263|
|DNH067877|    null|       null|     null|    null|      73160|
|SGH806190|    null|       null|     null|    null|      71306|
|HNH582022|    null|       null|     null|    null|    1188137|
|HNH795510|    null|      29937|      265|    null|     577161|
|DNFD91557|    null|       null|     null|    null|      31842|
+---------+--------+-----------+---------+--------+-----------+
only showing top 10 rows



In [ ]:
final = final.withColumnRenamed('Giải Trí','RelaxDuration').\
      withColumnRenamed('Phim Truyện','MovieDuration').\
          withColumnRenamed('Thiếu Nhi','ChildDuration').\
              withColumnRenamed('Thể Thao','SportDuration').\
                  withColumnRenamed('Truyền Hình','TVDuration')

In [ ]:
a = final.select('Contract','TVDuration').withColumn('Category',sf.lit('TV')).withColumnRenamed('TVDuration','TotalDuration')
b = final.select('Contract','RelaxDuration').withColumn('Category',sf.lit('Relax')).withColumnRenamed('RelaxDuration','TotalDuration')
c = final.select('Contract','MovieDuration').withColumn('Category',sf.lit('Movie')).withColumnRenamed('MovieDuration','TotalDuration')
d = final.select('Contract','ChildDuration').withColumn('Category',sf.lit('Child')).withColumnRenamed('ChildDuration','TotalDuration')
e = final.select('Contract','SportDuration').withColumn('Category',sf.lit('Sport')).withColumnRenamed('SportDuration','TotalDuration')
unpivot_data = a.union(b).union(c).union(d).union(e)

In [ ]:
from pyspark.sql.functions import rank
from pyspark.sql.window import Window
from pyspark.sql.functions import desc
from pyspark.sql.functions import collect_list

In [ ]:
windowSpec = Window.partitionBy('Contract').orderBy(desc('TotalDuration'))
most_watch = unpivot_data.withColumn('rank',rank().over(windowSpec))
most_watch = most_watch.filter(col('rank')==1)
most_watch = most_watch.withColumnRenamed('Category','Most_Watch')
most_watch = most_watch.select('Contract','most_watch')

In [ ]:
from pyspark.sql.types import FractionalType
caculate_taste = final.withColumn('RelaxDuration',when(final.RelaxDuration.isNotNull(),'Relax').otherwise(final.RelaxDuration))
caculate_taste = final.withColumn('MovieDuration',when(final.MovieDuration.isNotNull(),'Movie').otherwise(final.MovieDuration))
caculate_taste = final.withColumn('ChildDuration',when(final.ChildDuration.isNotNull(),'Child').otherwise(final.ChildDuration))
caculate_taste = final.withColumn('SportDuration',when(final.SportDuration.isNotNull(),'Sport').otherwise(final.SportDuration))
caculate_taste = final.withColumn('TVDuration',when(final.TVDuration.isNotNull(),'TV').otherwise(final.TVDuration))
caculate_taste = caculate_taste.withColumn('Taste', sf.concat_ws('-',*[c for c in caculate_taste.columns if c!='Contract']))
caculate_taste = caculate_taste.select('Contract','Taste')

In [ ]:
final = final.join(most_watch,'Contract','inner')
final = final.join(caculate_taste,'Contract','inner')

In [ ]:
#final.show(10)

+--------------+-------------+-------------+-------------+-------------+----------+----------+-------+
|      Contract|RelaxDuration|MovieDuration|ChildDuration|SportDuration|TVDuration|most_watch|  Taste|
+--------------+-------------+-------------+-------------+-------------+----------+----------+-------+
|             0|         null|         6127|         null|         null|6950351117|        TV|6127-TV|
|113.182.209.48|           89|         null|         null|         null|        63|     Relax|  89-TV|
|14.182.110.125|           92|         null|         null|         null|       404|        TV|  92-TV|
|222.252.254.74|         null|         null|         null|         null|       358|        TV|     TV|
|     AGAAA0338|         null|         null|         null|         null|    135831|        TV|     TV|
|     AGAAA0342|         null|         null|         null|         null|     56161|        TV|     TV|
|     AGAAA0346|         null|         null|         null|         null| 

In [ ]:
final.repartition(1).write.csv('/content/drive/MyDrive/result_log_content', header=True,mode="overwrite")

In [ ]:
url = 'jdbc:mysql://' + 'localhost' + ':' + '3306' + '/' + 'data_engineer'
driver = "com.mysql.cj.jdbc.Driver"
user = 'root'
password = ''

In [ ]:
#final.write.format('jdbc').option('url',url).option('driver',driver).option('dbtable','log_content').option('user',user).option('password',password).mode('append')